<a href="https://colab.research.google.com/github/Vish4github/Projects/blob/master/Sentiment_Classification_NB_%7C_Logistic_%7C_NNs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#%pip install fastai --upgrade

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import*
from fastai.text import*
from fastai.vision import *

In [0]:
import sklearn.feature_extraction.text as sklearn_text

In [0]:
#?? URLs

In [4]:
path = untar_data(URLs.IMDB_SAMPLE)
path

PosixPath('/root/.fastai/data/imdb_sample')

In [0]:
df=pd.read_csv(path/'texts.csv')

In [7]:
df.head()

,label,text,is_valid
0,negative,Un-bleeping-believable! Meg Ryan doesn't even ...,False
1,positive,This is a extremely well-made film. The acting...,False
2,negative,Every once in a long while a movie will come a...,False
3,positive,Name just says it all. I watched this movie wi...,False
4,negative,This movie succeeds at being one of the most u...,False


In [11]:
movie_reviews = (TextList.from_csv(path,'texts.csv',cols='text').split_from_df(col=2).label_from_df(cols=0))  #can help split data into test , validation etc - Here we are getting rid of col2 and saying col 0 is the label column

In [14]:
movie_reviews.valid.x[0],movie_reviews.valid.y[0]

(Text [ 2  5 21 71 ... 15  5  0 52], Category 1)

In [15]:
len(movie_reviews.train.x),len(movie_reviews.valid.x)

(800, 200)

In [17]:
len(movie_reviews.vocab.itos),len(movie_reviews.vocab.stoi)  #itos - integer to string and stoi - string to integer; the reason the nos are different - many words would be mapped as unknown, many as capital etc

(6008, 19159)

In [19]:
movie_reviews.vocab.itos[:10]  #based on frequency - what is itos - integer to strings - the movie reviews are stored as numbers(since the algo uses numbers and not strings) so if we want to see the actual words we need to use itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the']